In [9]:
from dotenv import load_dotenv

load_dotenv()

True

In [10]:
from langchain_community.document_loaders import JSONLoader

# Load data
loader = JSONLoader(
    file_path="../data/experience.json",
    jq_schema=".",
    text_content=False
)

# Load documents (this will be raw data from the JSON)
docs = loader.load()

In [11]:
import json
from langchain_core.documents import Document

documents = [
    Document(
        page_content=json.dumps(entry, ensure_ascii=False),
        metadata={
            "title": entry.get("title", ""),
            "start_date": entry.get("start_date", ""),
            "end_date": entry.get("end_date", ""),
            "association": entry.get("association", "")
        }
    )
    for entry in json.loads(docs[0].page_content)
]

In [12]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model="llama3:8b")

In [13]:
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="llama3:8b")

In [14]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)
_ = vector_store.add_documents(documents=documents)

In [16]:
from langchain import hub

# Define prompt for question-answering
prompt = hub.pull("rlm/rag-prompt")

In [19]:
from typing_extensions import List, TypedDict

# Define state for application
class State(TypedDict):
  question: str
  context: List[Document]
  answer: str

In [20]:
# Define application steps
def retrieve(state: State):
  retrieved_docs = vector_store.similarity_search(state["question"])
  return {"context": retrieved_docs}

def generate(state: State):
  docs_content = "\n\n".join(doc.page_content for doc in state["context"])
  messages = prompt.invoke({"question": state["question"], "context": docs_content})
  response = llm.invoke(messages)
  return {"answer": response.content}

In [22]:
from langgraph.graph import START, StateGraph

# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [26]:
response = graph.invoke({"question": "What experience did Andrew gain at GDSC?"})
print(response["answer"])

Andrew gained experience at GDSC (Google Developer Student Club) as a Frontend Developer from October to December 2024. He was part of the product cohort, one of two software developers selected out of 200 applicants, and attended hands-on workshops to improve communication with cross-functional teams and stakeholders.
